# Mediation analysis

In [2]:
library(brms)

Loading required package: Rcpp

Loading 'brms' package (version 2.18.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar




## Preliminaries

In [1]:
N <- 3000
M <- 2
K <- 4
J <- 3

mu <- 0
al <- matrix(c(-1, 1), ncol = 1)
Z <- matrix(rnorm(M*N, sd = .5), ncol = N)
ep_x <- matrix(rnorm(N, sd = 0.1), ncol = N)
X <- t(al) %*% Z + ep_x

U <- matrix(rnorm(K*N, sd = .2), ncol = N)
W <- sweep(U, 2, X, "+")

h_1 <- function(x) -x
h_2 <- function(x) 3*x
f <- function(x) 2*x
eta <- rnorm(N)
S <- f(X) + h_1(Z[1, ]) + h_2(Z[2, ]) + eta
S <- matrix(S, ncol = N)

beta0 <- matrix(rep(0, J*N), ncol = N)
beta1 <- matrix(rep(1, J), ncol = 1)
ep_y <- matrix(rnorm(J * N, sd = 0.1), ncol = N)
Y <- beta0 + beta1 %*% S + ep_y

data <- as.data.frame(t(rbind(Z,X,W,Y)))
names(data) <- c("Z1", "Z2", "X","W1", "W2", "W3", "W4", "Y1", "Y2", "Y3")

data$Xm <- as.numeric(NA)
data$Sm <- as.numeric(NA)



## Models

In [4]:
bf2 <- bf(Sm|mi() ~ mi(Xm) + Z1 + Z2)
bf3 <- bf(Y1 ~ 0 + mi(Sm))
bf4 <- bf(Y2 ~ 0 + mi(Sm))
bf5 <- bf(Y3 ~ 0 + mi(Sm))
bf8 <- bf(Xm|mi() ~ Z1 + Z2)
bf9 <- bf(W1 ~ 0 + mi(Xm))
bf10 <- bf(W2 ~ 0 + mi(Xm))
bf11 <- bf(W3 ~ 0 + mi(Xm))
bf12 <- bf(W4 ~ 0 + mi(Xm))

priors <- c(
    prior(normal(-1, 1), class = b, coef = Z1, resp = Sm),
    prior(normal(3, 1), class = b, coef = Z2, resp = Sm),
    prior(normal(2, 1), class = b, coef = miXm, resp = Sm),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y1),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y2),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y3),
    prior(normal(-1, 1), class = b, coef = Z1, resp = Xm),
    prior(normal(1, 1), class = b, coef = Z2, resp = Xm),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W1),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W2),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W3),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W4)
)

model <- brm(bf2 + bf3 + bf4 + bf5 + bf8 + bf9 + bf10 + bf11 + bf12 +set_rescor(FALSE), 
data = data, prior = priors, iter = 4000, cores = 4)

Compiling Stan program...

Start sampling

Warning message:
“The largest R-hat is 3.04, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#tail-ess”


In [5]:
bf2 <- bf(Sm|mi() ~ mi(Xm) + Z1 + Z2)
bf3 <- bf(Y1 ~ 0 + mi(Sm) + Z1 + Z2)
bf4 <- bf(Y2 ~ 0 + mi(Sm) + Z1 + Z2)
bf5 <- bf(Y3 ~ 0 + mi(Sm) + Z1 + Z2)
bf8 <- bf(Xm|mi() ~ Z1 + Z2)
bf9 <- bf(W1 ~ 0 + mi(Xm))
bf10 <- bf(W2 ~ 0 + mi(Xm))
bf11 <- bf(W3 ~ 0 + mi(Xm))
bf12 <- bf(W4 ~ 0 + mi(Xm))

priors <- c(
    prior(normal(-1, 1), class = b, coef = Z1, resp = Sm),
    prior(normal(3, 1), class = b, coef = Z2, resp = Sm),
    prior(normal(2, 1), class = b, coef = miXm, resp = Sm),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y1),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y2),
    prior(normal(1, 1), class = b, coef = miSm, resp = Y3),
    prior(normal(-1, 1), class = b, coef = Z1, resp = Xm),
    prior(normal(1, 1), class = b, coef = Z2, resp = Xm),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W1),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W2),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W3),
    prior(normal(1, 0.0001), class = b, coef = miXm, resp = W4), 
    prior(normal(0, 1), class = b, coef = Z1, resp = Y1),
    prior(normal(0, 1), class = b, coef = Z2, resp = Y1),
    prior(normal(0, 1), class = b, coef = Z1, resp = Y2),
    prior(normal(0, 1), class = b, coef = Z2, resp = Y2),
    prior(normal(0, 1), class = b, coef = Z1, resp = Y3),
    prior(normal(0, 1), class = b, coef = Z2, resp = Y3)
)

model2 <- brm(bf2 + bf3 + bf4 + bf5 + bf8 + bf9 + bf10 + bf11 + bf12 +set_rescor(FALSE), 
data = data, prior = priors, iter = 4000, cores = 4)

Compiling Stan program...

Start sampling

Warning message:
“There were 4000 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 3.74, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may b

In [6]:
summary(model2)$fixed

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”


,Estimate,Est.Error,l-95% CI,u-95% CI,Rhat,Bulk_ESS,Tail_ESS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Sm_Intercept,-0.0093126598,2.403631e-02,-0.064471820,0.030982951,1.258442,11.163973,34.50435
Xm_Intercept,0.0003931555,2.570327e-03,-0.004588283,0.005517164,1.000157,6094.207875,6585.00212
Sm_Z1,0.2018723645,9.764043e-01,-1.289233657,2.533596080,1.648392,6.475710,19.75418
Sm_Z2,0.4725509744,1.261916e+00,-1.353400707,2.450316462,2.071102,5.283832,26.24836
Y1_Z1,-0.6720048682,8.427992e-01,-2.225550137,1.074034277,1.704197,6.347270,20.25099
Y1_Z2,1.8092922671,1.564453e+00,-0.461399358,4.464777936,2.401055,4.900448,22.66333
Y2_Z1,-0.6772156630,8.403604e-01,-2.229117139,1.067843209,1.704323,6.347037,19.92750
Y2_Z2,1.8163689453,1.560094e+00,-0.441870868,4.466305556,2.400766,4.900674,22.50891
Y3_Z1,-0.6785239896,8.433648e-01,-2.231313003,1.068832649,1.704125,6.347531,19.84945


In [7]:
summary(model2)$spec_pars

Warning message:
“Parts of the model have not converged (some Rhats are > 1.05). Be careful when analysing the results! We recommend running more iterations and/or setting stronger priors.”


,Estimate,Est.Error,l-95% CI,u-95% CI,Rhat,Bulk_ESS,Tail_ESS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
sigma_Sm,0.94774378,0.474202763,0.44374678,2.2472008,3.1111148,4.512049,11.38072
sigma_Y1,0.10011570,0.002026959,0.09611526,0.1040494,1.0001002,8548.229345,5909.66226
sigma_Y2,0.10009910,0.002069000,0.09614182,0.1042503,1.0005697,8219.713321,5621.21304
sigma_Y3,0.09828170,0.002025848,0.09439626,0.1022895,0.9997260,7678.217924,6514.06741
sigma_Xm,0.09930972,0.002595589,0.09418763,0.1043857,1.0039903,2081.706540,3689.39664
sigma_W1,0.19573649,0.002946327,0.19003305,0.2016260,1.0003373,8117.368110,6304.42133
sigma_W2,0.19611900,0.002902856,0.19053123,0.2019865,1.0002413,8040.403229,6031.58718
sigma_W3,0.19844395,0.002938518,0.19265792,0.2043469,1.0003512,8576.438634,6320.32840
sigma_W4,0.19814526,0.003003409,0.19228536,0.2041068,0.9999083,8992.349159,6191.51830


## Mediation for original model

### Using the `bayestestR` package

In [11]:
install.packages("bayestestR")
library(bayestestR)

m1 <- mediation(model)

Installing package into ‘/Users/nescoba/Library/R/arm64/4.2/library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘insight’, ‘datawizard’





The downloaded binary packages are in
	/var/folders/gb/chvh9g355qqf4q5rw11d1m55xdk3f1/T//RtmpjVgcvg/downloaded_packages


ERROR: Error in if (variable %in% colnames(mf)) {: the condition has length > 1


In [12]:
m1 <- mediation(model2, treatment = "Z1", mediator = "Sm", outcome = "Y1")

ERROR: Error in .subset2(x, i, exact = exact): recursive indexing failed at level 2



In [13]:
m1 <- mediation(model, treatment = "Z1", mediation = "Xm", response = "Sm")

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in get1index


In [14]:
m1 <- mediation(model)

ERROR: Error in if (variable %in% colnames(mf)) {: the condition has length > 1


In [15]:
m1 <- mediation(model, response = "Sm")

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in get1index


In [16]:
m1 <- mediation(model, treatment = "Z1", mediation = "miXm", response = "Sm")

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in get1index


In [17]:
m1 <- mediation(model, treatment = "Z1", response = c(Xm = "miXm", "Sm"))

ERROR: Error in `[.data.frame`(insight::get_parameters(model), c(coef_indirect, : undefined columns selected


In [18]:
m1 <- mediation(model, treatment = "Z1", response = c(Xm = "miXm", Sm = "miSm"))

ERROR: Error in if (variable %in% colnames(mf)) {: argument is of length zero


In [19]:
m1 <- mediation(model, treatment = "Z1", response = c(miXm = "Xm", miSm = "Sm"))

ERROR: Error in if (variable %in% colnames(mf)) {: the condition has length > 1


In [20]:
names(insight::find_response(model, combine = TRUE))

[1] "Sm" "Y1" "Y2" "Y3" "Xm" "W1" "W2" "W3" "W4" "Sm" "Y1" "Y2" "Y3" "Xm" "W1"
[16] "W2" "W3" "W4"

In [21]:
m1 <- mediation(model, treatment = "Z1", response = c(Sm = "Sm"))

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in get1index


In [22]:
insight::get_parameters(model)

b_Sm_Intercept,b_Sm_Z1,b_Sm_Z2,bsp_Sm_miXm,sigma_Sm,bsp_Y1_miSm,sigma_Y1,bsp_Y2_miSm,sigma_Y2,bsp_Y3_miSm,⋯,b_Xm_Z2.1,sigma_Xm.1,bsp_W1_miXm.1,sigma_W1.1,bsp_W2_miXm.1,sigma_W2.1,bsp_W3_miXm.1,sigma_W3.1,bsp_W4_miXm.1,sigma_W4.1
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.0091141096,-0.2446136,1.372547,1.4992080,0.5749250,1.742508,0.09902868,1.740455,0.10157703,1.742042,⋯,0.9894173,0.10049621,0.9998507,0.1933146,1.0000061,0.1914069,0.9999393,0.1965938,0.9999134,0.1980532
0.0192413923,-0.4015140,1.606707,1.2733181,0.5759861,1.742520,0.09825962,1.742753,0.10246786,1.742515,⋯,0.9914083,0.09764157,0.9999013,0.1950061,1.0003347,0.1874089,1.0000369,0.1960657,1.0000542,0.1995207
0.0014242763,-0.4892473,1.664421,1.2068760,0.5871842,1.747338,0.09972973,1.745612,0.09984290,1.747345,⋯,0.9816321,0.09927816,0.9999229,0.1961226,1.0003265,0.1995112,1.0000179,0.2006636,1.0000457,0.1981186
0.0140844917,-0.8067995,1.952051,0.9063171,0.5864414,1.745028,0.10019805,1.744418,0.10070894,1.744759,⋯,0.9893785,0.10262490,0.9999446,0.1952738,1.0003425,0.1913862,1.0001751,0.2012594,1.0001984,0.2017655
-0.0072836107,-0.7266051,1.893401,0.9892694,0.5931883,1.741977,0.10197321,1.741160,0.09952891,1.741273,⋯,0.9851509,0.10008531,1.0000481,0.1918957,0.9999451,0.1918873,0.9998326,0.2028188,0.9999618,0.1958290
0.0246802737,-0.5104554,1.670651,1.1947524,0.5720278,1.740288,0.10059692,1.739296,0.10201831,1.740221,⋯,0.9869987,0.09857913,0.9997968,0.1991435,0.9999658,0.1937714,1.0001174,0.1969991,0.9999068,0.1957394
0.0049744229,-0.4437174,1.601330,1.2906698,0.5767442,1.739956,0.10078698,1.738455,0.10102300,1.738911,⋯,0.9881628,0.10453567,0.9999068,0.1937722,0.9999825,0.1948244,1.0001083,0.1999379,0.9999343,0.1973081
0.0174964526,-0.7128993,1.867462,0.9929390,0.5871397,1.739984,0.09995295,1.738347,0.10013111,1.741059,⋯,0.9865103,0.10204328,1.0002089,0.1928613,1.0001396,0.1955670,0.9999059,0.1986334,1.0001027,0.1976609
0.0111848563,-0.6405633,1.811547,1.0494408,0.5865574,1.740952,0.10019603,1.738973,0.09682565,1.739121,⋯,0.9949695,0.09795452,0.9999457,0.1974302,1.0000072,0.1944454,1.0000908,0.1993173,0.9998270,0.1963066


In [23]:
m1 <- mediation(model, treatment = "Z1", response = c(Xm = "miXm",Sm = "Sm"))

ERROR: Error in `[.data.frame`(insight::get_parameters(model), c(coef_indirect, : undefined columns selected


In [24]:
m1 <- mediation(model, treatment = "Z1", response = c(Xm = "miXm",Sm = "sp_Sm"))

ERROR: Error in if (variable %in% colnames(mf)) {: argument is of length zero


In [25]:
m1 <- mediation(model, treatment = "Z1", response = c(Xm = "miXm",Sm = "miSm"))

ERROR: Error in if (variable %in% colnames(mf)) {: argument is of length zero


In [28]:
print(names(insight::get_parameters(model)))

 [1] "b_Sm_Intercept"   "b_Sm_Z1"          "b_Sm_Z2"          "bsp_Sm_miXm"     
 [5] "sigma_Sm"         "bsp_Y1_miSm"      "sigma_Y1"         "bsp_Y2_miSm"     
 [9] "sigma_Y2"         "bsp_Y3_miSm"      "sigma_Y3"         "b_Xm_Intercept"  
[13] "b_Xm_Z1"          "b_Xm_Z2"          "sigma_Xm"         "bsp_W1_miXm"     
[17] "sigma_W1"         "bsp_W2_miXm"      "sigma_W2"         "bsp_W3_miXm"     
[21] "sigma_W3"         "bsp_W4_miXm"      "sigma_W4"         "b_Sm_Intercept.1"
[25] "b_Sm_Z1.1"        "b_Sm_Z2.1"        "bsp_Sm_miXm.1"    "sigma_Sm.1"      
[29] "bsp_Y1_miSm.1"    "sigma_Y1.1"       "bsp_Y2_miSm.1"    "sigma_Y2.1"      
[33] "bsp_Y3_miSm.1"    "sigma_Y3.1"       "b_Xm_Intercept.1" "b_Xm_Z1.1"       
[37] "b_Xm_Z2.1"        "sigma_Xm.1"       "bsp_W1_miXm.1"    "sigma_W1.1"      
[41] "bsp_W2_miXm.1"    "sigma_W2.1"       "bsp_W3_miXm.1"    "sigma_W3.1"      
[45] "bsp_W4_miXm.1"    "sigma_W4.1"      


In [27]:
m1 <- mediation(model, treatment = "Z1", mediator = "Xm", response = c(Xm = "miXm",Sm = "Sm"))

ERROR: Error in .subset2(x, i, exact = exact): subscript out of bounds


### Doing it myself

In [33]:
samples <- posterior_samples(model, pars = c("b_Sm_Z1", "bsp_Sm_miXm", "b_Xm_Z1"))
samples

Warning message:
“Method 'posterior_samples' is deprecated. Please see ?as_draws for recommended alternatives.”


b_Sm_Z1,bsp_Sm_miXm,b_Xm_Z1
<dbl>,<dbl>,<dbl>
-0.2446136,1.4992080,-1.0063325
-0.4015140,1.2733181,-1.0051004
-0.4892473,1.2068760,-1.0050129
-0.8067995,0.9063171,-1.0082990
-0.7266051,0.9892694,-1.0122453
-0.5104554,1.1947524,-0.9989742
-0.4437174,1.2906698,-1.0022937
-0.7128993,0.9929390,-1.0072755
-0.6405633,1.0494408,-1.0093951


In [34]:
?as_draws

draws-brms                package:brms                 R Documentation

_T_r_a_n_s_f_o_r_m '_b_r_m_s_f_i_t' _t_o '_d_r_a_w_s' _o_b_j_e_c_t_s

_D_e_s_c_r_i_p_t_i_o_n:

     Transform a ‘brmsfit’ object to a format supported by the
     ‘posterior’ package.

_U_s_a_g_e:

     ## S3 method for class 'brmsfit'
     as_draws(x, variable = NULL, regex = FALSE, inc_warmup = FALSE, ...)
     
     ## S3 method for class 'brmsfit'
     as_draws_matrix(x, variable = NULL, regex = FALSE, inc_warmup = FALSE, ...)
     
     ## S3 method for class 'brmsfit'
     as_draws_array(x, variable = NULL, regex = FALSE, inc_warmup = FALSE, ...)
     
     ## S3 method for class 'brmsfit'
     as_draws_df(x, variable = NULL, regex = FALSE, inc_warmup = FALSE, ...)
     
     ## S3 method for class 'brmsfit'
     as_draws_list(x, variable = NULL, regex = FALSE, inc_warmup = FALSE, ...)
     
     ## S3 method for class 'brmsfit'
     as_draws_rvars(x, variable = N

In [35]:
effect_indirect <- samples$b_Xm_Z1 * samples$bsp_Sm_miXm
effect_total <- samples$b_Sm_Z1 + effect_indirect
prop_mediated <- effect_indirect / effect_total

summary(prop_mediated)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4050  0.6538  0.7168  0.7173  0.7804  1.0595 

In [2]:
c_X_Z1 <- -1
c_S_X <- 2
c_S_Z1 <- -1

eff_ind <- c_X_Z1 * c_S_X
eff_tot <- c_S_Z1 + eff_ind
prop_med <- eff_ind / eff_tot

prop_med

[1] 0.6666667

## Mediation additional model

In [37]:
print(names(insight::get_parameters(model2)))

 [1] "b_Sm_Intercept"   "b_Sm_Z1"          "b_Sm_Z2"          "bsp_Sm_miXm"     
 [5] "sigma_Sm"         "b_Y1_Z1"          "b_Y1_Z2"          "bsp_Y1_miSm"     
 [9] "sigma_Y1"         "b_Y2_Z1"          "b_Y2_Z2"          "bsp_Y2_miSm"     
[13] "sigma_Y2"         "b_Y3_Z1"          "b_Y3_Z2"          "bsp_Y3_miSm"     
[17] "sigma_Y3"         "b_Xm_Intercept"   "b_Xm_Z1"          "b_Xm_Z2"         
[21] "sigma_Xm"         "bsp_W1_miXm"      "sigma_W1"         "bsp_W2_miXm"     
[25] "sigma_W2"         "bsp_W3_miXm"      "sigma_W3"         "bsp_W4_miXm"     
[29] "sigma_W4"         "b_Sm_Intercept.1" "b_Sm_Z1.1"        "b_Sm_Z2.1"       
[33] "bsp_Sm_miXm.1"    "sigma_Sm.1"       "b_Y1_Z1.1"        "b_Y1_Z2.1"       
[37] "bsp_Y1_miSm.1"    "sigma_Y1.1"       "b_Y2_Z1.1"        "b_Y2_Z2.1"       
[41] "bsp_Y2_miSm.1"    "sigma_Y2.1"       "b_Y3_Z1.1"        "b_Y3_Z2.1"       
[45] "bsp_Y3_miSm.1"    "sigma_Y3.1"       "b_Xm_Intercept.1" "b_Xm_Z1.1"       
[49] "b_Xm_Z2.1"        "sig

In [40]:
samples <- posterior_samples(model2, c("b_Y1_Z1", "bsp_Y1_miSm", "b_Sm_Z1"))
samples

Warning message:
“Method 'posterior_samples' is deprecated. Please see ?as_draws for recommended alternatives.”


b_Y1_Z1,bsp_Y1_miSm,b_Sm_Z1
<dbl>,<dbl>,<dbl>
-2.336900,-0.3732393,-1.5094339
-2.341948,-0.3737813,-1.7389181
-2.342563,-0.3734399,-1.6711928
-2.339548,-0.3730859,-1.7581337
-2.310850,-0.3738745,-1.5218329
-2.298000,-0.3761827,-0.9304456
-2.308006,-0.3759968,-2.1277447
-2.279015,-0.3768368,-1.9926585
-2.277312,-0.3768077,-1.8213606


In [41]:
effect_indirect <- samples$bsp_Y1_miSm * samples$b_Sm_Z1
effect_total <- samples$b_Y1_Z1 + effect_indirect
prop_mediated <- effect_indirect / effect_total

summary(prop_mediated)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-73.3570  -0.4363   0.1575   0.2330   0.8164  79.7249 